# Relatório e Projeto - Sistema de indexação e busca de documentos

## 1 - Informações da equipe
---
IN1152 - Recuperação Inteligente de Informação - 2022.2

**Equipe**: Matheus Rodrigues de Souza Félix (matheusrdgsf@gmail.com) e Rodrigo Souza de Melo (rsm5@cin.ufpe.br)

**Projeto "Sistema de indexação e busca de documentos"**

Professora Flavia de Almeida Barros (fab@cin.ufpe.br)

- Este relatório apresentará as seções solicitadas no Trabalho 1 - Sistema de indexação e busca de documentos. E mais abaixo o código-fonte do projeto

---

## 2 - Descrição dos documentos (corpus) que serão indexados pelo sistema
---

 - 2 Temas/tópicos dos documentos da sua base
 - Mostrar no relatório 2 ou 3 exemplos de documentos do corpus
 
 - BEIR [1] é um benchmark heterogêneo contendo diversas tarefas de RI. Ele também fornece uma estrutura comum e fácil para avaliação de seus modelos de recuperação baseados em PNL. Dentre as opções do BEIR foi selecionado o SciFact.
 - SciFact [4]: Devido ao rápido crescimento da literatura científica, há a necessidade de sistemas automatizados para auxiliar pesquisadores e o público na avaliação da veracidade das afirmações científicas. Para facilitar o desenvolvimento de sistemas para essa tarefa, é utilizado o SciFact, um conjunto de dados de 1,4 mil declarações escritas por especialistas, combinadas com resumos contendo evidências anotados com rótulos e justificativas de veracidade.
 - Leaderboard [4]: Avalia submissões de modelos no conjunto de dados SciFact, com o objetivo de desenvolver sistemas automatizados para verificação de alegações científicas.
 - Dataset [2]: Formando pelos elementos abaixo.
   - corpus: Representa o título e o texto do documento
   - query: Representa a consulta
   - qrels: Representa a relevância das consulta(s) realizada(s)
   
 - Exemplo de uma estrutura do documento com 2 queries, 2 documentos e as relevâncias de cada consulta [2].

~~~json
corpus = {
    "doc1" : {
        "title": "Albert Einstein", 
        "text": "Albert Einstein was a German-born theoretical physicist. who developed the theory of relativity, \
                 one of the two pillars of modern physics (alongside quantum mechanics). His work is also known for \
                 its influence on the philosophy of science. He is best known to the general public for his massâ€“energy \
                 equivalence formula E = mc2, which has been dubbed 'the world's most famous equation'. He received the 1921 \
                 Nobel Prize in Physics 'for his services to theoretical physics, and especially for his discovery of the law \
                 of the photoelectric effect', a pivotal step in the development of quantum theory."
        },
    "doc2" : {
        "title": "", # Keep title an empty string if not present
        "text": "Wheat beer is a top-fermented beer which is brewed with a large proportion of wheat relative to the amount of \
                 malted barley. The two main varieties are German WeiÃŸbier and Belgian witbier; other types include Lambic (made\
                 with wild yeast), Berliner Weisse (a cloudy, sour beer), and Gose (a sour, salty beer)."
    },
}

queries = {
    "q1" : "Who developed the mass-energy equivalence formula?",
    "q2" : "Which beer is brewed with a large proportion of wheat?"
}

qrels = {
    "q1" : {"doc1": 1},
    "q2" : {"doc2": 1},
}
~~~
 
 Referências: 
 * [1] https://github.com/beir-cellar/beir
 * [2] https://huggingface.co/datasets/BeIR/scifact-generated-queries
 * [3] https://github.com/allenai/scifact
 * [4] https://leaderboard.allenai.org/scifact/submissions/about
              

---

## 3 - Arquitetura do sistema 
---

- Prover uma descrição breve das etapas de processamento do sistema construído com base na ferramenta escolhida. Informar qual é o modelo de RI implementado pelo seu sistema, qual a fórmula para cálculo dos pesos e qual a função de ranking (vejam as aulas de modelos de RI). 

---

## 4 - Criação das bases de documentos indexados 
---

- Preparação & Indexação dos documentos: O sistema deve criar, de forma automática, cinco BASES indexadas a partir da base original de documentos. Cada BASE (no Solr, são COREs) deve utilizar processos diferentes na preparação (pré-processamento) dos dados. O objetivo é verificar qual é a melhor configuração de pré-processamento para o seu caso.

   - BASE 1: documentos originais, sem nenhum pré-processamento extra (só tokenização);
   - BASE 2: apenas eliminar stopwords (usar filtro de stopwords);
   - BASE 3: apenas usar stemming, sem eliminar as stopwords (não usar filtro de stopwords);
   - BASE 4: eliminar stopwords (usar filtro de stopwords) e usar stemming;
   - BASE 5: eliminar stopwords (usar filtro de stopwords), não usar stemming, e usar dicionário de sinônimos.


---


## 5 - Criação das consultas e Criação da Matriz de relevância  
---

- Prover uma descrição breve das etapas de processamento do sistema construído com base na ferramenta escolhida. Informar qual é o modelo de RI implementado pelo seu sistema, qual a fórmula para cálculo dos pesos e qual a função de ranking (vejam as aulas de modelos de RI). 

- Esta etapa foi descrita em mais detalhes na atividade correspondente, postada no Classroom. 
  ===> DÚVIDA: NÃO ENCONTREI OS DETALHES NO CLASSROOM.
  
- Mostrar aqui parte da matriz - basta mostrar as 5 colunas exemplificadas abaixo (incluindo a coluna final com a quantidade de documentos relevantes). 

Exemplo de Matriz de relevância “Consultas x Documentos”.


|                                                                      | Doc 1 | Doc 2 | ... Doc 20 | Qtd de docs relevantes |   |   |   |   |   |
|----------------------------------------------------------------------|-------|-------|------------|------------------------|---|---|---|---|---|
| Consulta1 Ex.:  como faço o agendamento para tomar vacina de covid ? | 1     | 0     | 1          | 10                     |   |   |   |   |   |
| Consulta2 Ex.: Qual o preço de carro FIAT UNO usado ?                     | 1     | 1     | 0          | 15                     |   |   |   |   |   |




---

## 6 - Testes/Avaliação 
---

- Submeter as 2 consultas para cada BASE criada, e avaliar cada resultado separadamente -  i.e., calcular separadamente a precisão e a cobertura de cada consulta em relação a cada BASE criada. 
- Usar as fórmulas vistas em aula: precisão, cobertura e F-measure. 
- Incluir no relatório uma matriz de resultados para CADA consulta. Assim podemos ver a influência do pré-processamento dos documentos no resultado final do sistema.





Matriz de resultados para a Consulta 1
- Consulta: incluam aqui o texto da consulta avaliada nessa matriz
- Qtd de documentos relevantes: ver matriz de relevância (avaliação manual)

|  | **Precisão** | **Cobertura** | **F-measure** | **Qtd de Docs relevantes retornados pela  consulta 1** | **Qtd total de documentos retornados pela consulta 1** |
|:--------:|:------------:|:-------------:|:-------------:|:------------------------------------------------------:|:------------------------------------------------------:|
| BASE 1   |              |               |               |                                                        |                                                        |
| BASE 2   |              |               |               |                                                        |                                                        |
| BASE 3   |              |               |               |                                                        |                                                        |
| BASE 4   |              |               |               |                                                        |                                                        |
| BASE 5   |              |               |               |                                                        |                                                        |





Matriz de resultados para a Consulta 2
- Consulta: incluam aqui o texto da consulta avaliada nessa matriz
- Qtd de documentos relevantes: ver matriz de relevância (avaliação manual)

|  | **Precisão** | **Cobertura** | **F-measure** | **Qtd de Docs relevantes retornados pela  consulta 2** | **Qtd total de documentos retornados pela consulta 2** |
|:--------:|:------------:|:-------------:|:-------------:|:------------------------------------------------------:|:------------------------------------------------------:|
| BASE 1   |              |               |               |                                                        |                                                        |
| BASE 2   |              |               |               |                                                        |                                                        |
| BASE 3   |              |               |               |                                                        |                                                        |
| BASE 4   |              |               |               |                                                        |                                                        |
| BASE 5   |              |               |               |                                                        |                                                        |




Matriz de resultados para o Sistema
- As medidas de precisão, cobertura e F-meause do sistema serão obtidas calculando-se a média entre os resultados obtidos com cada consulta em relação a cada BASE criado.


|        | Precisão média | Cobertura média | F-measure média |
|--------|----------------|-----------------|-----------------|
| BASE 1 |                |                 |                 |
| BASE 2 |                |                 |                 |
| BASE 3 |                |                 |                 |
| BASE 4 |                |                 |                 |
| BASE 5 |                |                 |                 |


---

 

## 7 - Conclusão 
---

- Deve conter um texto curto explicando o que vocês concluíram a partir do resultado dos testes (tabelas acima).

---

## Código-fonte do projeto

### Configurações

In [1]:
#!pip install GitPython

### Libs

In [98]:
import os
import pandas as pd
from git import Repo
import gzip
from io import BytesIO
import gzip

from sklearn.metrics import confusion_matrix, classification_report #Utilizado para calcular a matrix de confusão e Relatório de classificação - Checar outra forma para remover o warning e não utilizar o parâmetro zero_division=1 no recall_score 

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import unidecode
import string
import re

nltk.download('punkt')
nltk.download('stopwords')

from sklearn.feature_extraction.text import TfidfVectorizer

from scipy import spatial
from sklearn.neighbors import NearestNeighbors

from sklearn.metrics import precision_score, f1_score, recall_score

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Matheus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Matheus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Data Collect Scifact

In [55]:
# DATA_PATH = "dataset/scifact-generated-queries"

In [56]:
# if not os.path.isdir(DATA_PATH):
#     Repo.clone_from("https://huggingface.co/datasets/BeIR/scifact-generated-queries", "dataset/scifact-generated-queries")

In [57]:
# with gzip.open(DATA_PATH+"/train.jsonl.gz", 'rb') as f:
#     file_content = f.read()

In [58]:
# dataset = pd.read_json(BytesIO(file_content), lines=True)

In [99]:
# dataset.groupby(["title"]).size().reset_index(name='queries_count')

### Data Collect 20News Labeled

In [100]:
DATA_PATH = "20news_labeled.csv"

In [136]:
dataset = pd.read_csv(DATA_PATH)
dataset.dropna(inplace=True)

In [137]:
dataset.head()

_id  \
0    0   
1    1   
2    2   
3    3   
4    4   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

## Pre-rocesing Data

In this step we will create 5 data models:

v1 - Only Tokenization; \
v2 - Only Stopword Filter; \
v3 - Only Stemming; \
v4 - Remove Stopwords and Stemming; \
v5 - Remove Stopworpd and expand words with Synonyms.


In [184]:
def remove_accent(text):
    return unidecode.unidecode(text)

def tokenize(text):
    return word_tokenize(text, language="english")

def pre_process(text, rmv_sw, stem):
    text_lower = text.lower()
    text_rmv_accent = remove_accent(text_lower)
    text_tokenized = tokenize(text_rmv_accent)
    text_rmv_alphanum = list(filter(lambda token: not re.search('\d', token), text_tokenized))
    text_final = list(filter(lambda token: token not in string.punctuation+, text_rmv_alphanum))
    
    if rmv_sw:
        text_final = list(filter(lambda token: token not in STOPWORDS, text_final))
    
    if stem:
        stemmer = SnowballStemmer("english")
        text_final = list(map(lambda token: stemmer.stem(token), text_final))
        
    return text_final

STOPWORDS = set(map(lambda token: remove_accent(token), stopwords.words("english")))

In [185]:
pre_process_v1 = lambda text: pre_process(text, False, False)
pre_process_v2 = lambda text: pre_process(text, True, False)
pre_process_v3 = lambda text: pre_process(text, False, True)
pre_process_v4 = lambda text: pre_process(text, True, True)
#pre_process_v5 = lambda i: pre_process(text, True, False) # TODO: Para este caso eu chequei esta referência -> https://www.holisticseo.digital/python-seo/nltk/wordnet.

In [186]:
dataset_pcrs = dataset.copy().drop_duplicates().head(50)

In [187]:
dataset_pcrs["v1"] = dataset_pcrs["text"].apply(lambda text: pre_process_v1(text))

In [188]:
dataset_pcrs["v2"] = dataset_pcrs["text"].apply(lambda text: pre_process_v2(text))

In [189]:
dataset_pcrs["v3"] = dataset_pcrs["text"].apply(lambda text: pre_process_v3(text))

In [190]:
dataset_pcrs["v4"] = dataset_pcrs["text"].apply(lambda text: pre_process_v4(text))

In [191]:
# @TODO: Create Expand Vocabulary Pipeline add connect pre_process
# dataset_pcrs["v5"] = dataset_pcrs["text"].apply(lambda text: pre_process_v5(text))

In [192]:
# Be careful with dataset_pcrs Len
#HTML(dataset_pcrs.head(3).to_html())

# @TODO: add "()" to STOPWORDS. A better strategy is create a dict simbol to filter and add this pipeline.
dataset_pcrs.head(2)

_id  \
0    0   
1    1   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

### Vectorizing Data

This step defines 5 (vectorizer, corpus_vetorized) models.

In [195]:
def get_documments(df_column):
    return list(map(lambda tokenized_text: " ".join(tokenized_text), df_column))

In [196]:
corpus_v1 = get_documments(dataset_pcrs["v1"])
corpus_v2 = get_documments(dataset_pcrs["v2"])
corpus_v3 = get_documments(dataset_pcrs["v3"])
corpus_v4 = get_documments(dataset_pcrs["v4"])
# corpus_v5 = get_documments(dataset_pcrs["v5"])

In [197]:
vectorizer_v1 = TfidfVectorizer()
corpus_v1_vct = vectorizer_v1.fit_transform(corpus_v1)

vectorizer_v2 = TfidfVectorizer()
corpus_v2_vct = vectorizer_v2.fit_transform(corpus_v2)

vectorizer_v3 = TfidfVectorizer()
corpus_v3_vct = vectorizer_v3.fit_transform(corpus_v3)

vectorizer_v4 = TfidfVectorizer()
corpus_v4_vct = vectorizer_v4.fit_transform(corpus_v4)

# vectorizer_v5 = TfidfVectorizer()
# corpus_v1_vct = vectorizer_v5.fit_transform(corpus_v5)

### Retrieval Information Function

In [198]:
# kd -> KDTree, nn -> Nearest Neighbor, bf -> Brute Force
# n -> number of docs in return, use -1 to all docs
def info_retrieval(pre_process, corpus, vectorizer, query, n=2, matcher="kd"):
    
    query = " ".join(pre_process(query))
    query_vct = vectorizer.transform([query])
    
    if n == -1:
        
        n = corpus.shape[0]
    
    if matcher == "kd":
        
        kdtree = scipy.spatial.KDTree(corpus_v1_vct.todense())
        
        # p is Minkowski p-norm.
        # p = 1, Manhattan Distance
        # p = 2, Euclidean Distance
        # p = +inf, Chebychev Distance
        distance, index = kdtree.query(query_vct.todense(), n, p=1)
        
    elif matcher == "nn":
        
        nbrs = NearestNeighbors(n_neighbors=n, algorithm="ball_tree").fit(corpus)
        distance, index = nbrs.kneighbors(query_vct)
        
    elif matcher == "bf":

        nbrs = NearestNeighbors(n_neighbors=n, algorithm="brute", metric="cosine").fit(corpus)
        distance, index = nbrs.kneighbors(query_vct)
        
    else:
        
        return "Matcher strategy not avaliable. Set kd to KDTree, nn to Nearest Neighbor and bf to Brute Force"
    
    return list(zip(distance.tolist()[0], index.tolist()[0]))

In [199]:
ir_v1 = lambda query: info_retrieval(pre_process_v1, corpus_v1_vct, vectorizer_v1, query, n=-1, matcher = "bf")
ir_v2 = lambda query: info_retrieval(pre_process_v2, corpus_v2_vct, vectorizer_v2, query, n=-1, matcher = "bf")
ir_v3 = lambda query: info_retrieval(pre_process_v3, corpus_v3_vct, vectorizer_v3, query, n=-1, matcher = "bf")
ir_v4 = lambda query: info_retrieval(pre_process_v4, corpus_v4_vct, vectorizer_v4, query, n=-1, matcher = "bf")
# ir_v5 = lambda query: info_retrieval(pre_process_v5, corpus_v5_vct, vectorizer_v5, query, n=-1, matcher = "bf")

### Snippet Test

In [203]:
query_1 = "Moral and ethics in christianism"
query_2 = 

In [204]:
result = sorted(ir_v1(query), key=lambda i:i[0])

In [205]:
result

[(0.8390608686972042, 3),
 (0.8991347213458132, 22),
 (0.9260732872630356, 5),
 (0.9310174352872853, 17),
 (0.931373084223062, 14),
 (0.9369893113510906, 24),
 (0.9393816132858184, 10),
 (0.9398635625822763, 26),
 (0.9434762214855852, 1),
 (0.9465797970445144, 0),
 (0.9479106325345472, 16),
 (0.9576016806628308, 18),
 (0.9576714605795643, 6),
 (0.9595431028822449, 25),
 (0.9596341593129576, 13),
 (0.9615343319101327, 4),
 (0.9635281658333272, 21),
 (0.9638897268442101, 23),
 (0.9650987409714858, 20),
 (0.974025461722243, 15),
 (0.9755987102116255, 27),
 (0.9761505116142708, 9),
 (0.9762709742374461, 11),
 (0.9812412103759419, 19),
 (0.9888929260057168, 7),
 (1.0, 8),
 (1.0, 2),
 (1.0, 12)]

In [208]:
# Nearest Documment to Query Snippet
# pd.set_option('display.max_colwidth', None)

result_nn = result[0][1]
dataset_pcrs[["_id", "text"]].iloc[[result_nn]]

In [214]:
dataset_pcrs[["_id", "text"]].iloc[[result_nn]]

,_id,text
3,3,"See, there you go again, saying that a moral act is only significantif it is ""voluntary."" Why do you think this?And anyway, humans have the ability to disregard some of their instincts.You are attaching too many things to the term ""moral,"" I think.Let's try this: is it ""good"" that animals of the same speciesdon't kill each other. Or, do you think this is right? Or do you think that animals are machines, and that nothing they dois either right nor wrong?Those weren't arbitrary killings. They were slayings related to some sortof mating ritual or whatnot.Yes it was, but I still don't understand your distinctions. Whatdo you mean by ""consider?"" Can a small child be moral? How abouta gorilla? A dolphin? A platypus? Where is the line drawn? Doesthe being need to be self aware?What *do* you call the mechanism which seems to prevent animals ofthe same species from (arbitrarily) killing each other? Don'tyou find the fact that they don't at all significant?"


### Evaluation

In [236]:
# Modeling Query x Relavance
docs = ["doc_%d"%i for i in range(len(dataset))]
_data = list(zip(dataset.q1.tolist(), dataset.q2.tolist()))
_dict = {docs[i]: _data[i] for i in range(len(dataset))}
query_relevance = pd.DataFrame(data=_dict, index = ["Q%d"%(i+1) for i in range(len(_data[0]))])